In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import itertools
from gensim.models.word2vec import Text8Corpus
from gensim.corpora import Dictionary
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

/home/zqtan/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


We load the `text8` corpus. It is one very long succession of words. There are no sentence boundaries or punctuation marks.

In [2]:
with open('./text8') as f:
    s = f.read()
s[:300]

' anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organiz'

To split the corpus into sentences, it is convenient to use the `Text8Corpus` function from `gensim`. We choose to work with $1000$ words per sentence.

In [3]:
corpus = Text8Corpus('./text8', max_sentence_length=1000)
sentences = list(itertools.islice(Text8Corpus('./text8', max_sentence_length=1000), None))
sentences[0]


['anarchism',
 'originated',
 'as',
 'a',
 'term',
 'of',
 'abuse',
 'first',
 'used',
 'against',
 'early',
 'working',
 'class',
 'radicals',
 'including',
 'the',
 'diggers',
 'of',
 'the',
 'english',
 'revolution',
 'and',
 'the',
 'sans',
 'culottes',
 'of',
 'the',
 'french',
 'revolution',
 'whilst',
 'the',
 'term',
 'is',
 'still',
 'used',
 'in',
 'a',
 'pejorative',
 'way',
 'to',
 'describe',
 'any',
 'act',
 'that',
 'used',
 'violent',
 'means',
 'to',
 'destroy',
 'the',
 'organization',
 'of',
 'society',
 'it',
 'has',
 'also',
 'been',
 'taken',
 'up',
 'as',
 'a',
 'positive',
 'label',
 'by',
 'self',
 'defined',
 'anarchists',
 'the',
 'word',
 'anarchism',
 'is',
 'derived',
 'from',
 'the',
 'greek',
 'without',
 'archons',
 'ruler',
 'chief',
 'king',
 'anarchism',
 'as',
 'a',
 'political',
 'philosophy',
 'is',
 'the',
 'belief',
 'that',
 'rulers',
 'are',
 'unnecessary',
 'and',
 'should',
 'be',
 'abolished',
 'although',
 'there',
 'are',
 'differing',
 '

As done in the paper we only keep the $8000$ most frequent words.

In [4]:
dct = Dictionary(corpus)
dct.filter_extremes(no_below=1, no_above=0.5, keep_n=8000)  #筛选词频
#dct.doc2idx('anarchism')

We establish a correspondence between words and unique numeric identifiers. This makes it easy to build one-hot vectors afterwards.

In [5]:
sents2idx = [dct.doc2idx(sentence) for sentence in sentences]         #编号是按照字母序对单词排的（可以通过打印看）
sents2idx = [[idx for idx in sent if idx != -1] for sent in sents2idx]
sents_8k = [[dct[idx] for idx in sent] for sent in sents2idx]
dct2 = Dictionary(sents_8k)
sents2idx = [dct.doc2idx(sentence) for sentence in sents_8k]

Here is what the first sentence of the corpus looks like (first 10 words only):

In [6]:
sents2idx[0][:10]

[193, 261, 1, 11, 83, 303, 53, 133, 90, 233]

We pre-compute all the context windows for all the sentences in the corpus.

In [7]:
contexts_all = []
for sent in sents2idx:
    contexts_sent = []
    for i in range(len(sent)):
        context = []
        for j in range(max(i-2,0), min(i+2, len(sent))):
            context.append(sent[j])
        contexts_sent.append((sent[i], context))
    contexts_all.append(contexts_sent)

Here is what the contexts for the first 10 words of the first sentence look like.

In [8]:
contexts_all[0][:10]

[(193, [193, 261]),
 (261, [193, 261, 1]),
 (1, [193, 261, 1, 11]),
 (11, [261, 1, 11, 83]),
 (83, [1, 11, 83, 303]),
 (303, [11, 83, 303, 53]),
 (53, [83, 303, 53, 133]),
 (133, [303, 53, 133, 90]),
 (90, [53, 133, 90, 233]),
 (233, [133, 90, 233, 111])]

We define a custom dataloader that creates the batches during training. Each batch is made of `bsize` sentences.

In [9]:
def loader(bsize):
    idx_shuffle = np.random.permutation(len(sents2idx))
    for i in range(0, len(sents2idx), bsize):
        batch = []
        for idx in idx_shuffle[i: min(i+bsize, len(sents2idx))]:
            contexts = contexts_all[idx]
            sent = sents2idx[idx]
            contexts_enriched = []
            for (token, context) in contexts:
                negatives = []
                for _ in range(len(context)):
                    negative = random.choice(sent)  # Negative sampling happens here
                    while negative in context:
                        negative = random.choice(sent)
                    negatives.append(negative)
                contexts_enriched.append((token, context, negatives))
            batch.append(contexts_enriched)
        yield batch
                

Here is the code for the Sinkhorn iterations. We chose to implement vanilla iterations, but it would have been preferable to work in log-space.

In [10]:





mydevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#mydevice="cpu"
        
#alpha = 2.0
               
n_inner = 10

def smoothabs(x, alpha):
    """
    differential absolute function using smooth max.

    Caution.
    if x is large (over 100), this returns nan.

    Parameters
    ----------
    x : torch.tensor
    
    Returns
    ----------
    result : torch.tensor

    """
    return  (x*torch.exp(alpha*x) - x*torch.exp(-alpha*x)) / (2.0 + torch.exp(alpha*x) + torch.exp(-alpha*x)).to(mydevice)
 


class Tree_dis(torch.nn.Module):

    def __init__(self, n_inner, n_leaf, device='cpu', max_d=None, alpha=2.0):
        super(Tree_dis, self).__init__()
        self.n_inner = n_inner # number of inner nodes which contained root.
        self.n_leaf = n_leaf # number of leaf nodes. (e.g., number of words)
        self.n_node = self.n_leaf + self.n_inner # number of all nodes.
        self.device = device
        self.alpha = alpha
               
        self.param = torch.nn.Parameter(torch.randn(self.n_inner, self.n_leaf, device=self.device)) # D2

        # initialize parameters
        nn.init.normal_(self.param, 0.0, 0.1)
        
        self.A = self.gen_A()
        self.inv_A = self.calc_inv_A() # (I - D1)^{-1}

    def gen_A(self):
        """
        Initialize D1, which is an adjacency matrix of a tree consisting of internal nodes and return I - D1.
        Assume D1 is the perfect 5-ary tree.
        """
        
        A = torch.zeros(self.n_inner, self.n_inner-1)
        for i in range(1, int(self.n_inner/5)):
            A[i-1, 5*(i-1):5*i] = 1.0
        A[int(self.n_inner/5)-1, 5*(int(self.n_inner/5)-1):] = 1.0
        A = A.to(self.device)
        return torch.eye(self.n_inner, device=self.device) - torch.cat([torch.zeros(self.n_inner, 1, device=self.device), A], dim=1).to(self.device)

    
    def calc_inv_A(self):
        """
        return (I - D1)^{-1}.
        """
        return self.A.inverse()

    
    def calc_ppar(self):
        """
        return upper two blocks of D_par.
        """
        
        exp_param = F.softmax(self.param, dim=0)
        return torch.cat([torch.eye(self.n_inner, device=self.device) - self.A, exp_param], dim=1)

    
    def calc_psub(self, block_D=None):
        """
        Retuens
        ----------
        X : torch.tensor (shape is (self.n_inner, self.n_leaf))
            X[i][j] is P_sub (v_j+self.n_inner | v_i).
        """
        
        B = F.softmax(self.param, dim=0).to(mydevice)
        X = torch.mm(self.inv_A, B).to(mydevice)
        return X.to(mydevice)

    
    def calc_distance(self, mass1, mass2, block_psub=None):
        """
        Parameters
        ----------
        mass1 : torch.tensor (shape is (self.n_leaf))
            normalized bag-of-words.
        mass2 : torch.tensor (shape is (self.n_leaf))
            normalized bag-of-words
        block_psub : torch.tensor (shape is (self.n_inner, self.n_leaf))
            retuen value of self.calc_plock_psub().
        """
        if block_psub is None:
            block_psub = self.calc_psub()
        return (smoothabs(torch.mv(block_psub, mass1 - mass2), alpha=self.alpha).sum() + smoothabs(mass1 - mass2, alpha=self.alpha).sum()).to(mydevice)

    def forward(self, x , y):
        return self.calc_distance(x,y).to(mydevice)



We define the architecture for the neural network. The inputs are one-hot vectors of size $8000$ and the output is a $2M$ vector that will be reshaped as a cloud of $M$ points in $\mathbb R^2$.

In [11]:
n_leaf = 4  #M为leaf数

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8000, 64)
        self.fc2 = nn.Linear(64, n_leaf)
        #self.param=torch.nn.Parameter(torch.randn(n_inner, n_leaf ,device=mydevice))   #D2
        #self.param=torch.randn(n_inner, n_leaf )
        self.soft= nn.Softmax()
        #self.X= calc_psub(self.param)
        self.relu = nn.ReLU()
        

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.soft(x)
        return x


net = Net()
net.to(mydevice)
tree_loss = Tree_dis(n_inner,n_leaf,mydevice)


In [12]:
optimizer = optim.Adam([obj for obj in net.parameters()]+[obj for obj in tree_loss.parameters()] ,lr=10**-4)
#itertools.chain(net.parameters(),tree_loss.parameters())

In [13]:
def one_hot(n):
    vect = torch.zeros(8000)
    vect[n] = 1.0
    return vect.to(mydevice)

Next is our training loop. Please disregard the output of this cell. The network we trained was saved as `net.pth`. The training was done with a batch size of $1$ and we interrupted it after $4$ hours (the first epoch was far from over since the loop went through only $\sim 3000$ sentences).

In [ ]:
m = 1
bsize = 1
epochs = 2



for epoch in range(epochs):

    running_loss = 0.0
    for i, batch in enumerate(loader(bsize), 0):   #每个batch做一次参数优化（即一次正向反向）
        optimizer.zero_grad()

        loss = torch.zeros(1).to(mydevice)
        #print("starting batch")
        for sent in batch:
            for (token, context, negatives) in sent:
                token_emb = net(one_hot(token)) 
                for positive in context:
                    positive_emb = net(one_hot(positive)) 
                    
                    loss += tree_loss(token_emb,positive_emb) **2
                for negative in negatives:
                    negative_emb =  net(one_hot(negative)) 
                    loss += net.relu(m-tree_loss(token_emb,negative_emb)) **2
            #print("finished sentence")
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 40==39:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss))
            running_loss = 0.0

print('Finished Training')

In [ ]:
torch.save(net, './net.pth')

In [ ]:
net = torch.load('./net.pth')
net.eval()

In [18]:
with torch.no_grad():
    test1 = net(one_hot(dct2.token2id['power']))
    test2 = net(one_hot(dct2.token2id['earth']))
    test3 = net(one_hot(dct2.token2id['mathematics']))
    test4 = net(one_hot(dct2.token2id['preservation']))
print('power',net.soft(test1))
print('earth',net.soft(test2))
print('mathematics',net.soft(test3))
print('preservation',net.soft(test4))

power tensor([0.1975, 0.2534, 0.2074, 0.3417], device='cuda:0')
earth tensor([0.1900, 0.2195, 0.2025, 0.3880], device='cuda:0')
mathematics tensor([0.1956, 0.3287, 0.2009, 0.2748], device='cuda:0')
preservation tensor([0.1919, 0.3673, 0.1953, 0.2456], device='cuda:0')


<ipython-input-12-d89db8fd0b42>:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.soft(x)
<ipython-input-18-f81320553feb>:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print('power',net.soft(test1))
<ipython-input-18-f81320553feb>:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print('earth',net.soft(test2))
<ipython-input-18-f81320553feb>:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print('mathematics',net.soft(test3))
<ipython-input-18-f81320553feb>:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print('preservation',net.soft(test4))


In [19]:
with torch.no_grad():
    test1 = net(one_hot(dct2.token2id['football']))
    test2 = net(one_hot(dct2.token2id["basketball"]))
    test3 = net(one_hot(dct2.token2id['soccer']))
    test4 = net(one_hot(dct2.token2id['rugby']))
print('football',net.soft(test1))
print("basketball",net.soft(test2))
print('soccer',net.soft(test3))
print('rugby',net.soft(test4))



football tensor([0.1916, 0.3751, 0.1948, 0.2385], device='cuda:0')
basketball tensor([0.1948, 0.2866, 0.2026, 0.3160], device='cuda:0')
soccer tensor([0.1948, 0.2944, 0.2022, 0.3085], device='cuda:0')
rugby tensor([0.1942, 0.3363, 0.1992, 0.2702], device='cuda:0')


<ipython-input-12-d89db8fd0b42>:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.soft(x)
<ipython-input-19-172cbd652841>:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print('football',net.soft(test1))
<ipython-input-19-172cbd652841>:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print("basketball",net.soft(test2))
<ipython-input-19-172cbd652841>:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print('soccer',net.soft(test3))
<ipython-input-19-172cbd652841>:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print('rugby',net.soft(test4))


In [26]:
for obj in tree_loss.parameters():
    print(obj)           #没softmax前的，但也可以大小直接看

Parameter containing:
tensor([[ 0.1048, -0.2223,  0.1390, -0.0582],
        [ 0.1683,  0.1879, -0.3083, -0.1569],
        [-0.1339, -0.3543,  0.2439,  0.1495],
        [ 0.1014, -0.1270,  0.2683,  0.3706],
        [ 0.0021, -0.1617,  0.2419,  0.1417],
        [-0.0322, -0.2477,  0.2202,  0.0320],
        [-0.2538, -0.1566,  0.0391,  0.2050],
        [-0.1332,  0.3028, -0.2484, -0.0427],
        [ 0.0675,  0.3515, -0.1485, -0.2209],
        [ 0.1382,  0.3066, -0.2694, -0.3281]], device='cuda:0',
       requires_grad=True)


In [4]:
len(sents2idx)

NameError: name 'sents2idx' is not defined